## What is SQLAlchemy

SQLAlchemy is a dtabase toolkit and an ORM (object-relational mapper) implementation written in Python, and provides a generalized interface for creating and executing database-agnostic code without needing to write SQL statements.

A benefit many developers enjoy with SQLAlchemy is that it allows them to write Python code in their project to map from the database schema to the application's Python objects. No SQL is required to create, maintain and query the database. The mapping allows SQLAlchemy to handle the underlying database so developers can work with their Python objects instead of writing bridge code to get data in and out of relational tables. This ORM feature is however mostly used in web applications such as Flask, while in this tutorial we will be using the Core features of SQLAlchemy.

SQLAlchemy currently supports 5 types of SQL databases: SQLite, MySQL and PostgreSQL, Oracle and Firebird.

If you need a graphical program to manage your databases, consider <a href="https://sqlitebrowser.org/" target="_blank">https://sqlitebrowser.org/</a> or <a href="https://dbeaver.io/" target="_blank">https://dbeaver.io/</a> for any operating system.

If the package doesn't exist in your system, just install it with: `pip install sqlalchemy` or `conda install sqlalchemy`, depending on your python distribution.

### Creating a database and connecting to it

In [66]:
# the create_engine() function will create a new file if it doesn't exist and do nothing if it does
from sqlalchemy import create_engine

# using relative path
engine = create_engine('sqlite:///example.db')

# using absolute path
#engine = create_engine('sqlite:////path/to/database.db') 

engine.connect()
 
print(engine)

Engine(sqlite:///example.db)


## Defining the database schema

### Simple table

Tables in SQLAlchemy are represented as an instance of the Table class. The Table constructor function accepts table name, metadata and one or more columns as arguments. Here is an example:

In [67]:
from sqlalchemy import MetaData, Table, String, Column, Text, DateTime, Boolean, Integer, Numeric, CheckConstraint, ForeignKey
from datetime import datetime

# the MetaData object holds all the information about the database and the tables it contains. 
# we use an instance of it to create or drop tables in the database.
metadata = MetaData()

users = Table('users', metadata,
    Column('id', Integer(), primary_key=True),
    Column('name', String(200), nullable=False),
)

items = Table('items', metadata,
    Column('id', Integer(), primary_key=True),
    Column('name', String(200), nullable=False),
    Column('cost_price', Numeric(10, 2), nullable=False),
    Column('selling_price', Numeric(10, 2),  nullable=False),
    Column('quantity', Integer(), nullable=False),
    CheckConstraint('quantity > 0', name='quantity_check')
)

Here's a quick comparison table for the most common data types between SQLAlchemy, Python and SQL:

<table>
<thead>
<tr><th>SQLAlchemy </th>
<th> Python </th>
<th> SQL</th>
</tr>
</thead>
<tbody>
<tr><td>BigInteger </td>
<td> int </td>
<td> BIGINT</td>
</tr>

<tr><td>Boolean </td>
<td> bool </td>
<td> BOOLEAN or SMALLINT</td>
</tr>

<tr><td>Date </td>
<td> datetime.date </td>
<td> DATE</td>
</tr>

<tr><td>DateTime </td>
<td> datetime.datetime </td>
<td> DATETIME</td>
</tr>

<tr><td>Integer </td>
<td> int </td>
<td> INTEGER</td>
</tr>

<tr><td>Float </td>
<td> float </td>
<td> FLOAT or REAL</td>
</tr>

<tr><td>Numeric </td>
<td> decimal.Decimal </td>
<td> NUMERIC</td>
</tr>

<tr><td>Text </td>
<td> str </td>
<td> VARCHAR or TEXT</td>
</tr>
</tbody></table>

### One-to-Many relationship

In [68]:
# we create a new table, adding the field that will contain the relationship: user_id
posts = Table('posts', metadata,
    Column('id', Integer(), primary_key=True),
    Column('post_title', String(200), nullable=False),
    Column('post_slug', String(200),  nullable=False),
    Column('content', Text(),  nullable=False),
    Column('published', Boolean(),  default=False),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now),
    Column('user_id', ForeignKey("users.id"))
)

### One-to-One relationship

In [69]:
# we create a new table, setting the relationship between the users table and this one.
user_details = Table('user_details', metadata,
    Column('id', Integer(), primary_key=True),                 
    Column('user_id', ForeignKey('users.id'), unique=True), # the relationship field. Notice the 'unique' option, ensuring the one-to-one relationship.
    Column('picture', String(200), nullable=False),
    Column('url', String(200), nullable=False),
    Column('twitter_profile', String(200), nullable=False)  
)

### Many-to-Many relationship

In [70]:
tags = Table('tags', metadata,
    Column('id', Integer(), primary_key=True),
    Column('tag', String(200), nullable=False),
    Column('tag_slug', String(200),  nullable=False),    
)
 
post_tags = Table('post_tags', metadata,
    Column('post_id', ForeignKey('posts.id')),
    Column('tag_id', ForeignKey('tags.id'))
)

### Submiting your schema to the database

The `create_all()` method only creates a table if it doesn’t already exist in the database. That means you can call `create_all()` safely multiple times. Note that calling `create_all()` method after changing the table definition will not alter the table schema. To do that, we can use a database migration tool called <a href="https://alembic.sqlalchemy.org/en/latest/" target="_blank">Alembic</a>.

In [71]:
metadata.create_all(engine)

### Inspecting your database schema

The `metadata` object is where all the information about the database lies. It has multiple properties that allow us to query our database.

In [72]:
# the tables property to list our tables
for table in metadata.tables:
    print(table)

users
items
posts
user_details
tags
post_tags


In [73]:
# once we have our table info, we can query it's details as well:
for table in metadata.tables:
    print(f'Information for table: {table}')
    table_obj = metadata.tables[table]
    print('-' * 40)
    print(table_obj.columns)         # return a list of columns
    print(table_obj.c)               # same as above
    print(table_obj.foreign_keys)    # returns a set containing foreign keys on the table
    print(table_obj.primary_key)     # returns the primary key of the table
    print(table_obj.metadata)        # get the MetaData object from the table
    print()

Information for table: users
----------------------------------------
['users.id', 'users.name']
['users.id', 'users.name']
set()
PrimaryKeyConstraint(Column('id', Integer(), table=<users>, primary_key=True, nullable=False))
MetaData(bind=None)

Information for table: items
----------------------------------------
['items.id', 'items.name', 'items.cost_price', 'items.selling_price', 'items.quantity']
['items.id', 'items.name', 'items.cost_price', 'items.selling_price', 'items.quantity']
set()
PrimaryKeyConstraint(Column('id', Integer(), table=<items>, primary_key=True, nullable=False))
MetaData(bind=None)

Information for table: posts
----------------------------------------
['posts.id', 'posts.post_title', 'posts.post_slug', 'posts.content', 'posts.published', 'posts.created_on', 'posts.updated_on', 'posts.user_id']
['posts.id', 'posts.post_title', 'posts.post_slug', 'posts.content', 'posts.published', 'posts.created_on', 'posts.updated_on', 'posts.user_id']
{ForeignKey('users.id')}
P

In [74]:
# we can go even further and get information about each column
for table in metadata.tables:
    if(table == 'posts'):
        print(f'Columns in table: {table}')
        table_obj = metadata.tables[table]
        print('-' * 40)
        for col in table_obj.columns:
            print(col.name)        # column name
            print(col.type)        # column type
            print(col.nullable)    # can it be null?
            print(col.primary_key) # is it a primary key?
            print(col.comment)     # print the comment
            print()
            
            

Columns in table: posts
----------------------------------------
id
INTEGER
False
True
None

post_title
VARCHAR(200)
False
False
None

post_slug
VARCHAR(200)
False
False
None

content
TEXT
False
False
None

published
BOOLEAN
True
False
None

created_on
DATETIME
True
False
None

updated_on
DATETIME
True
False
None

user_id
INTEGER
True
False
None



## CRUD (create, read, update, delete) operations

### Creating (inserting) records

There are several ways to insert records into the database. The most basic way is to use the `insert()` method of the Table instance and pass values of the columns as keyword arguments to the `values()` method.

In [75]:
ins = users.insert().values(
    name = 'John Doe'
)

In [76]:
# to inspect the generated sql:
print(str(ins))
print()

# to view the whole object being inserted
ins.compile().params

INSERT INTO users (name) VALUES (:name)



{'name': 'John Doe'}

In [77]:
# now we need to execute the query in the database
conn = engine.connect()
r = conn.execute(ins)
r

In [78]:
# this ResultProxy object has some nice attributes you can use:
# know the id of the inserted element
print(r.inserted_primary_key)
print()

# how many rows where inserted
print(r.rowcount)

[1]

1


#### Multiple inserts

In [79]:
# set the insert object
multiple_ins = users.insert()

# define the list that holds our records
user_list = [
    { 'name': 'Jane Doe' },
    { 'name': 'Peter Doe' },
    { 'name': 'Margaret Doe' }
]

# connect and execute
conn = engine.connect()
r = conn.execute(multiple_ins, user_list)

#total inserted rows
print(r.rowcount)
print()

3



### Reading (selecting) records

In [80]:
all_users = users.select()
print(str(all_users))

SELECT users.id, users.name 
FROM users


In [81]:
r = conn.execute(all_users)
r.fetchall()

[(1, 'John Doe'), (2, 'Jane Doe'), (3, 'Peter Doe'), (4, 'Margaret Doe')]

> **Note** : the `fetchall()` method loads all the results into the memory at once. Thus, it is not very efficient on a large result set. Alternatively, you can use for loop to iterate over the result set one at a time.

In [82]:
all_users = users.select()
r = conn.execute(all_users)
for row in r:
    print(row)

(1, 'John Doe')
(2, 'Jane Doe')
(3, 'Peter Doe')
(4, 'Margaret Doe')


Here is a list of some common methods and attributes of the `ResultProxy` object.

<table>
<thead>
<tr><th>Method/Attribute </th>
<th> Description</th>
</tr>
</thead>
<tbody>
<tr><td>fetchone()</td>
<td> fetch the next row from the result set. If the result set has been exhausted, subsequent calls to <code>fetchone()</code> returns <code>None</code>.</td>
</tr>

<tr><td>fetchmany(size=None)</td>
<td> fetch the specified number of rows from the result set. If the result set has been exhausted, subsequent calls to <code>fetchmany()</code> returns <code>None</code>.</td>
</tr>

<tr><td>fetchall()</td>
<td> fetch all the rows from the result set. If the result set has been exhausted, subsequent calls to <code>fetchall()</code> returns <code>None</code>.</td>
</tr>

<tr><td>first()</td>
<td> fetch the first row from the result set and close the connection. This means that after calling the <code>first()</code> method we can't access any other rows in the result set, until we send the query to the database again (using the <code>execute()</code> method).</td>
</tr>

<tr><td>keys()</td>
<td> returns a list of columns from where data is retrieved.</td>
</tr>
</tbody></table>

Let's try them out:

In [83]:
# fetchone()
r = conn.execute(all_users)
print(r.fetchone())
print(r.fetchone())
print(r.fetchone())

(1, 'John Doe')
(2, 'Jane Doe')
(3, 'Peter Doe')


In [84]:
# fetchmany()
r = conn.execute(all_users)
print(r.fetchmany()) # defaults to 1, same as first()
print()

print(r.fetchmany(3)) # same as LIMIT in SQL

[(1, 'John Doe')]

[(2, 'Jane Doe'), (3, 'Peter Doe'), (4, 'Margaret Doe')]


In [85]:
# first()
r = conn.execute(all_users)
print(r.first())

(1, 'John Doe')


In [86]:
# keys()
r = conn.execute(all_users)
print(r.keys())

['id', 'name']


#### Accessing the selected records

If you look at the data returned by `fetchXXX()`, you'll notice that it seems like a list of tuples, being each tuple a record. But that's actually not the case. It is a RowProxy object, which allows us to access data in the row using column name, index position or `Column` instance. For example:

In [88]:
r = conn.execute(all_users)
row = r.fetchone()
print(row)
print(type(row))
print(row['id'], row['name'])    # access column data via column name
print(row[0], row[1])    # access column data via column index position
print(row[users.c.id], row[users.c.name])    # access column data via Column object
print(row.id, row.name)    # access column data via attribute

(1, 'John Doe')
<class 'sqlalchemy.engine.result.RowProxy'>
1 John Doe
1 John Doe
1 John Doe
1 John Doe


#### Filtering the selection

Let's add data to the items table to show the available types of filters:

In [89]:
items_list = [
    {
        "name":"Chair",
        "cost_price": 9.21,
        "selling_price": 10.81,
        "quantity": 5
    },
    {
        "name":"Pen",
        "cost_price": 3.45,
        "selling_price": 4.51,
        "quantity": 3
    },
    {
        "name":"Headphone",
        "cost_price": 15.52,
        "selling_price": 16.81,
        "quantity": 50
    },
    {
        "name":"Travel Bag",
        "cost_price": 20.1,
        "selling_price": 24.21,
        "quantity": 50
    },
    {
        "name":"Keyboard",
        "cost_price": 20.12,
        "selling_price": 22.11,
        "quantity": 50
    },
    {
        "name":"Monitor",
        "cost_price": 200.14,
        "selling_price": 212.89,
        "quantity": 50
    },
    {
        "name":"Watch",
        "cost_price": 100.58,
        "selling_price": 104.41,
        "quantity": 50
    },
    {
        "name":"Water Bottle",
        "cost_price": 20.89,
        "selling_price": 25.00,
        "quantity": 50
    },
]

ins = items.insert()
r = conn.execute(ins, items_list)
r.rowcount

8

To filter records we use `where()` method. It accept a condition and adds a WHERE clause to the SELECT statement.

In [90]:
# this query will return all the items whose cost price is greater than 20
s = items.select().where(
    items.c.cost_price > 20
)

r = conn.execute(s)
r.fetchall()

/usr/lib/python3.7/site-packages/sqlalchemy/sql/sqltypes.py:666: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


[(4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

> See the warning above? It's something you should consider if you need precision in your work. We don't, so for now here's how to disable those warnings:

In [91]:
import warnings

with warnings.catch_warnings():
    # this query will return all the items whose cost price is greater than 20
    s = items.select().where(
    items.c.cost_price > 20
    )

r = conn.execute(s)
r.fetchall()

[(4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

In [92]:
import warnings

with warnings.catch_warnings():
    # we can specify additional conditions by simple chaining the where() method
    s = items.select().\
    where(items.c.cost_price + items.c.selling_price > 50).\
    where(items.c.quantity > 10)

r = conn.execute(s)
r.fetchall()

[(6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50)]

As you can see, when we chain the where() method the conditions are ANDed together.

So how do we specify OR or NOT conditions to our SELECT statement?

It turns out that instead of chaining where() method there are two other ways to combine conditions:

1. Bitwise Operators.
2. Conjunctions.

Let’s start with the first one.

**Bitwise Operators**

Bitwise Operators `&`, `|` and `~` allow us to connect conditions with SQL AND, OR and NOT operators respectively.

The preceding query can be coded using bitwise operators as follows:

In [94]:
import warnings

with warnings.catch_warnings():
    # using & (AND)
    s = items.select().\
    where(
        (items.c.cost_price + items.c.selling_price > 50) & 
        (items.c.quantity > 10)
    )

r = conn.execute(s)
r.fetchall()

[(6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50)]

In [95]:
import warnings

with warnings.catch_warnings():
    # using | (OR)
    s = items.select().\
    where(
        (items.c.cost_price > 200) |
        (items.c.quantity < 5)
    )

r = conn.execute(s)
r.fetchall()

[(2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50)]

In [97]:
import warnings

with warnings.catch_warnings():
    # using ~ (NOT)
    s = items.select().\
    where(
        ~(items.c.quantity == 50) &
        (items.c.cost_price < 20)
    )

r = conn.execute(s)
r.fetchall()

[(1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3)]

**Conjunctions**

Another way to connect conditions is to use conjunction functions i.e `and_()`, `or_()` and `not_()`.

> This is the preferred way of defining conditions in SQLAlchemy.

In [98]:
from sqlalchemy import and_, or_, not_
import warnings

with warnings.catch_warnings():
    # using and_()
    s = items.select().\
    where(    
        and_(
            items.c.quantity >= 50,
            items.c.cost_price < 100,
        )
    ) 

r = conn.execute(s)
r.fetchall()

[(3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

In [99]:
import warnings

with warnings.catch_warnings():
    # using and_() and not_() together
    s = items.select().\
    where(    
        and_(
            items.c.quantity >= 50,
            items.c.cost_price < 100,
            not_(
                items.c.name == 'Headphone'            
            ),        
        )
    ) 

r = conn.execute(s)
r.fetchall()

[(4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

**Other common comparison operators**

Below are examples for other comparison operators: `IS NULL`, `IS NOT NULL`, `IN`, `NOT IN`, `BETWEEN`, `NOT BETWEEN`, `LIKE` and `NOT LIKE`

In [101]:
import warnings

with warnings.catch_warnings():
    # testing IS NULL
    s = items.select().\
    where(    
        items.c.quantity == None
    ) 

r = conn.execute(s)
r.fetchall()

[]

In [102]:
import warnings

with warnings.catch_warnings():
    # testing IS NOT NULL
    s = items.select().\
    where(    
        items.c.quantity != None
    ) 

r = conn.execute(s)
r.fetchall()

[(1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

In [103]:
import warnings

with warnings.catch_warnings():
    # testing IN
    s = items.select().\
    where(    
        items.c.name.in_(["Monitor", "Watch"])
    ) 

r = conn.execute(s)
r.fetchall()

[(6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50)]

In [104]:
import warnings

with warnings.catch_warnings():
    # testing NOT IN
    s = items.select().\
    where(    
        items.c.name.notin_(["Monitor", "Watch"])
    ) 

r = conn.execute(s)
r.fetchall()

[(1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

In [105]:
import warnings

with warnings.catch_warnings():
    # testing BETWEEN
    s = items.select().\
    where(    
        items.c.quantity.between(1, 10)
    ) 

r = conn.execute(s)
r.fetchall()

[(1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3)]

In [106]:
import warnings

with warnings.catch_warnings():
    # testing NOT BETWEEN
    s = items.select().\
    where(    
        not_(items.c.quantity.between(1, 10))
    ) 

r = conn.execute(s)
r.fetchall()

[(3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

In [107]:
import warnings

with warnings.catch_warnings():
    # testing LIKE
    s = items.select().\
    where(    
        items.c.name.like('%ch%')
    ) 

r = conn.execute(s)
r.fetchall()

[(1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50)]

In [108]:
import warnings

with warnings.catch_warnings():
    # testing NOT LIKE
    s = items.select().\
    where(    
        not_(items.c.name.like('%ch%'))
    ) 

r = conn.execute(s)
r.fetchall()

[(2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50)]

#### Ordering the selection

The `order_by()` method adds ORDER BY clause to the SELECT statement. It accepts one or more columns to sort by. For each column listed in the `order_by()` clause, you can specify whether the rows are sorted in ascending order (using `asc()`) or descending order (using `desc()`). If neither, specified rows are sorted in ascending order. For example:

In [109]:
from sqlalchemy import desc, asc
import warnings

with warnings.catch_warnings():
    # sorts the rows first by quantity (in ascending order) and then by cost_price (in descending order)
    s = items.select().\
    order_by(
        items.c.quantity, 
        desc(items.c.cost_price)
    ) 

r = conn.execute(s)
r.fetchall()

[(2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50),
 (4, 'Travel Bag', Decimal('20.10'), Decimal('24.21'), 50),
 (3, 'Headphone', Decimal('15.52'), Decimal('16.81'), 50)]

#### Limiting the selection

The `limit()` method adds the LIMIT clause to the SELECT statement. It accepts an integer which indicates the number of rows to return. For example:

In [110]:
import warnings

with warnings.catch_warnings():
    # we only want the first four results
    s = items.select().\
    order_by(
        items.c.quantity, 
        desc(items.c.cost_price)
    ).limit(4) 

r = conn.execute(s)
r.fetchall()

[(2, 'Pen', Decimal('3.45'), Decimal('4.51'), 3),
 (1, 'Chair', Decimal('9.21'), Decimal('10.81'), 5),
 (6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50)]

To specify an offset (i.e. the starting position) to the LIMIT clause use the `offset()` method.

In [111]:
import warnings

with warnings.catch_warnings():
    # we only want the first four results after bypassing the first two
    s = items.select().\
    order_by(
        items.c.quantity, 
        desc(items.c.cost_price)
    ).limit(4).offset(2)

r = conn.execute(s)
r.fetchall()

[(6, 'Monitor', Decimal('200.14'), Decimal('212.89'), 50),
 (7, 'Watch', Decimal('100.58'), Decimal('104.41'), 50),
 (8, 'Water Bottle', Decimal('20.89'), Decimal('25.00'), 50),
 (5, 'Keyboard', Decimal('20.12'), Decimal('22.11'), 50)]

#### Selecting only some columns from the table

If you want to select only certain columns from the table, you have to import another `select` from the SQLAlchemy package, and slighly change your query, like the example below:

In [112]:
from sqlalchemy import select
import warnings

with warnings.catch_warnings():
    # s = items.select().\
    s = select([items.c.name, items.c.quantity]).\
    order_by(
        items.c.quantity, 
        desc(items.c.cost_price)
    ).limit(4).offset(2)

r = conn.execute(s)
r.fetchall()

[('Monitor', 50), ('Watch', 50), ('Water Bottle', 50), ('Keyboard', 50)]

#### Adding labels to the selection

Just as in SQL, we can perform simple calculations on the rows retrieved before sending them to the output. For example:

In [113]:
import warnings

with warnings.catch_warnings():
    s = select([
        items.c.name, 
        items.c.quantity, 
        items.c.selling_price * 5 
    ]).where(
    items.c.quantity ==  50
)

r = conn.execute(s)
# notice the new label automatically given to the calculation with selling_price
print(r.keys())

['name', 'quantity', 'anon_1']


You can assign our own labels to any column:

In [114]:
import warnings

with warnings.catch_warnings():
    s = select([
        items.c.name, 
        items.c.quantity, 
        # our new label
        (items.c.selling_price * 5).label('Price') 
    ]).where(
    items.c.quantity ==  50
)

r = conn.execute(s)
print(r.keys())

['name', 'quantity', 'Price']


#### Accessing native database functions

To access the built-in functions provided by the database we use func object. The following listing shows how to use some date/time, mathematical and string functions found in SQLite database.

In [115]:
from sqlalchemy.sql import func
 
c = [    
    # date/time functions
    func.date(),
    func.time(),
    func.current_timestamp(),      
    func.now(),
    
    # number functions    
    func.max(4,3,2),
    func.min(4,3,2),
    func.avg(4),
    func.count(),
    func.sum(3),
    
    # string functions   
    func.lower("ABC"),
    func.upper("abc"),
    func.length("abc"),
    func.trim("  ab c  "),    
    func.sqlite_version()
]

s = select(c)
r = conn.execute(s)
for row in r.fetchone():
    print(row)

2019-09-25
17:33:38
2019-09-25 17:33:38
2019-09-25 17:33:38
4
2
4.0
1
3
abc
ABC
3
ab c
3.29.0


In [116]:
c = [ 
    func.sum(items.c.quantity),
    func.avg(items.c.quantity),    
    func.max(items.c.quantity),
    func.min(items.c.quantity),
    func.count(items.c.id),    
]
 
s = select(c)
r = conn.execute(s)
for row in r.fetchone():
    print(row)

308
38.5
50
3
8


#### Grouping the selection

In [117]:
c = [ 
    func.count("*").label('count'),         
    items.c.name,
    items.c.quantity
]
 
s = select(c).group_by(items.c.quantity)
r = conn.execute(s)
r.fetchall()

[(1, 'Pen', 3), (1, 'Chair', 5), (6, 'Headphone', 50)]

#### Joining tables

The Table instance provides the following two methods to create joins:

1. `join()` – creates inner join
2. `outerjoin()` – creates outer join (LEFT OUTER JOIN to be specific)

The inner join returns only the rows which matches the join condition, whereas the outer join returns the rows which matches the join condition as well as some additional rows.

Both methods accept a `Table` instance, figures out the join condition based on the foreign key relationship and returns a JOIN construct.

Let's add some data to the posts table and experiment one join.

In [118]:
data = [
    {
        'post_title': 'Post 1',
        'post_slug': 'post-1',
        'content': 'Blah blah blah.',
        'published': True,
        'user_id': 1
    },
    {
        'post_title': 'Post 2',
        'post_slug': 'post-2',
        'content': 'Blah blah blah.',
        'published': True,
        'user_id': 2
    },
    {
        'post_title': 'Post 3',
        'post_slug': 'post-3',
        'content': 'Blah blah blah.',
        'published': True,
        'user_id': 1
    },
    {
        'post_title': 'Post 4',
        'post_slug': 'post-4',
        'content': 'Blah blah blah.',
        'published': True,
        'user_id': 1
    },
    {
        'post_title': 'Post 5',
        'post_slug': 'post-5',
        'content': 'Blah blah blah.',
        'published': True,
        'user_id': 3
    },
]

query = posts.insert()

conn = engine.connect()
r = conn.execute(query, data)
r.rowcount

5

**Join**

In [119]:
# join example: we want the pair post title and author name where the author has 'Doe' in it's name and it starts with 'Jonh'
s = select([
            posts.c.post_title,
            users.c.name
]).select_from(
    posts.join(users)
).where(
    and_(
        users.c.name.like("%Doe%"),
        users.c.name.like("John%")
    )
)

# here's the query in SQL:
print(s)
print()

# here's the results:
r = conn.execute(s)
print(r.fetchall())

SELECT posts.post_title, users.name 
FROM posts JOIN users ON users.id = posts.user_id 
WHERE users.name LIKE :name_1 AND users.name LIKE :name_2

[('Post 1', 'John Doe'), ('Post 3', 'John Doe'), ('Post 4', 'John Doe')]


Pay special atention to this line right here:

`FROM posts JOIN users ON users.id = posts.user_id`

Notice that SQLAlchemy has automatically infered the relationship fields (`users.id` and `posts.user_id`) based on the foreign keys we set up previously, since we haven't declared them in our script.

**Outer Join**

The `Table` instance we pass to the `outerjoin()` method is placed on the right side of the outer join. As a result, the below query will return all the rows from `users` table (the left table), and only the rows that meets the join condition are returned from the `posts` table (the right table).

In [120]:
s = select([        
    users.c.name,
    posts.c.post_title,
]).select_from(
    users.outerjoin(posts)
)
 
print(s)
print()

r = conn.execute(s)
for row in r.fetchall():
    print(row)

SELECT users.name, posts.post_title 
FROM users LEFT OUTER JOIN posts ON users.id = posts.user_id

('John Doe', 'Post 1')
('John Doe', 'Post 3')
('John Doe', 'Post 4')
('Jane Doe', 'Post 2')
('Peter Doe', 'Post 5')
('Margaret Doe', None)


**Full Outer Join**

Just as a side note, you can also create a FULL OUTER JOIN by passing `full=True` to the `outerjoin()` method. However, SQLite doesn't support it yet, so the following query will return an error:

In [121]:
s = select([        
    users.c.name,
    posts.c.post_title,
]).select_from(
    posts.outerjoin(users, full=True)
)
 
print(s)
print()

r = conn.execute(s)
for row in r.fetchall():
    print(row)

SELECT users.name, posts.post_title 
FROM posts FULL OUTER JOIN users ON users.id = posts.user_id



OperationalError: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT users.name, posts.post_title 
FROM posts FULL OUTER JOIN users ON users.id = posts.user_id]
(Background on this error at: http://sqlalche.me/e/e3q8)

#### Dealing with duplicates

To deal with the duplicate rows in the result set we use the DISTINCT option. We can add DISTINCT option to the SELECT statement using the `distinct()` method. For example, imagine that you want to know how many different quantities there are in the `items` table:

In [122]:
from sqlalchemy import distinct

s = select([distinct(items.c.quantity)])

r = conn.execute(s)
r.fetchall()

[(5,), (3,), (50,)]

So we know that we have 3 different quantities. But how much of each quantity do we have?

In [123]:
s = select([
    distinct(items.c.quantity),
    func.count(items.c.quantity)
]).group_by(items.c.quantity)

r = conn.execute(s)
r.fetchall()

[(3, 1), (5, 1), (50, 6)]

#### Raw Queries

SQLAlchemy also gives you the flexibility to execute raw SQL using the `text()` function. For example:

In [124]:
from sqlalchemy.sql import text
 
s = text(
"""
SELECT posts.post_title, users.name 
FROM posts JOIN users ON users.id = posts.user_id 
WHERE users.name LIKE :name_1 AND users.name LIKE :name_2
"""
)

# if you need to place variables in the query (ex: name_1), you can then set their values on the execute method
r = conn.execute(s, name_1="%Doe%", name_2='John%')
r.fetchall()

[('Post 1', 'John Doe'), ('Post 3', 'John Doe'), ('Post 4', 'John Doe')]

### Updating records

Updating records is achieved using the `update()` function. For example, the following query updates the selling_price and quantity of Water Bottle to 30 and 60, respectively.

In [125]:
from sqlalchemy import update
 
s = update(items).where(
    items.c.name == 'Water Bottle'
).values(
    selling_price = 30,
    quantity = 60,
)

r = conn.execute(s)
print(r.rowcount)  # total updated rows
print()

# just to make sure it did update the record:
print(conn.execute(select([items.c.selling_price, items.c.quantity]).where(items.c.name == 'Water Bottle')).fetchall())

1

[(Decimal('30.00'), 60)]


### Deleting records

To delete records we use the `delete()` function. Let's duplicate our users and then delete them:

In [ ]:
multiple_ins = users.insert()

user_list = [
    { 'name': 'John Doe' },
    { 'name': 'Jane Doe' },
    { 'name': 'Peter Doe' },
    { 'name': 'Margaret Doe' }
]

r = conn.execute(multiple_ins, user_list)

print(conn.execute(select([users])).fetchall())

In [ ]:
from sqlalchemy import delete

s = delete(users).where(
    users.c.id.in_([5,6,7,8])
)

conn.execute(s)

print(conn.execute(select([users])).fetchall())

### Clearing the database schema

We can drop all the tables in the database using `MetaData.drop_all()` method.

In [126]:
metadata.drop_all(engine)